Pair programming A/B testing

Ejercicio 1:
Supongamos que estamos realizando un experimento para determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").

Objetivo del ejercicio: Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.

In [30]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [31]:
df= pd.read_csv("marketing_AB.csv", index_col=0)
df

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14
...,...,...,...,...,...,...
588096,1278437,ad,False,1,Tuesday,23
588097,1327975,ad,False,1,Tuesday,23
588098,1038442,ad,False,3,Tuesday,23
588099,1496395,ad,False,1,Tuesday,23


In [32]:
df.columns

Index(['user id', 'test group', 'converted', 'total ads', 'most ads day',
       'most ads hour'],
      dtype='object')

In [33]:
df.isnull().sum()

user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

In [34]:
df.duplicated().sum()

0

In [35]:
df.describe()

,user id,total ads,most ads hour
count,5.881010e+05,588101.000000,588101.000000
mean,1.310692e+06,24.820876,14.469061
std,2.022260e+05,43.715181,4.834634
min,9.000000e+05,1.000000,0.000000
25%,1.143190e+06,4.000000,11.000000
50%,1.313725e+06,13.000000,14.000000
75%,1.484088e+06,27.000000,18.000000
max,1.654483e+06,2065.000000,23.000000


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 27.5+ MB


In [37]:
df["user id"].unique()

array([1069124, 1119715, 1144181, ..., 1038442, 1496395, 1237779])

In [38]:
df['test group'].unique()

array(['ad', 'psa'], dtype=object)

In [39]:
df.sample (30)

,user id,test group,converted,total ads,most ads day,most ads hour
543269,1586402,ad,False,2,Monday,13
158264,1539194,ad,False,5,Monday,11
52678,1374321,ad,False,11,Thursday,12
299934,1379498,ad,False,20,Friday,19
384069,1113490,ad,False,22,Thursday,18
358783,1576973,ad,False,2,Thursday,15
199234,1235720,ad,False,17,Tuesday,19
330969,1047482,ad,False,10,Sunday,14
58043,1491615,ad,False,147,Wednesday,6
385032,1082508,ad,False,2,Thursday,16


In [40]:
columnas_nuevas=df['test group'].str.split(' ', expand= True)
columnas_nuevas

,0
0,ad
1,ad
2,ad
3,ad
4,ad
...,...
588096,ad
588097,ad
588098,ad
588099,ad


In [41]:
group_ad = []
group_psa = []

for elmto in df['test group']:
    group_ad.append('ad')
    group_psa.append('psa')

df['group_ad'] = group_ad
df['group_psa'] = group_psa



df.drop('test group', axis=1, inplace=True)

In [42]:
df

,user id,converted,total ads,most ads day,most ads hour,group_ad,group_psa
0,1069124,False,130,Monday,20,ad,psa
1,1119715,False,93,Tuesday,22,ad,psa
2,1144181,False,21,Tuesday,18,ad,psa
3,1435133,False,355,Tuesday,10,ad,psa
4,1015700,False,276,Friday,14,ad,psa
...,...,...,...,...,...,...,...
588096,1278437,False,1,Tuesday,23,ad,psa
588097,1327975,False,1,Tuesday,23,ad,psa
588098,1038442,False,3,Tuesday,23,ad,psa
588099,1496395,False,1,Tuesday,23,ad,psa
